# High Performance/Throughput Computing

In [1]:
def f(x):
    y = 0.0
    for i in range(100000):
        y += x
    return y

In [2]:
f(0.01)

999.9999999992356

In [3]:
%timeit f(0.01)

100 loops, best of 3: 6.5 ms per loop


## Cython

[Cython](http://cython.org/) ist ein Compiler,
der Cython Code in C/C++ Code übersetzt und mit der Python C-Bibliothek verbindet.
Dies eignet sich hervorragend dazu,
um langsame Teile eines größeren Python Programmes selektiv zu beschleunigen.
Dies wird dadurch erreicht, dass einzelne Funktionen oder Klassen in die Sprache "*Cython*" umgeschrieben werden.
Dies ist nicht besonders schwierig, da Cython eine Obermenge einer Untermenge der Python Sprache ist:
das heißt, einige komplexere Programmstrukturen gibt es nicht, dafür aber Erweiterungen wie z.B. native Typenbezeichnungen.

Die einzige wirkliche Herausforderung ist,
genau zu verstehen welche Datentypen im Spiel sind und
wie aufwändig die eventuelle Konvertierung der einzelnen Objekte zwischen C/C++ und Python ist.
In der [Dokumentation](http://docs.cython.org/) gibt es dazu viele Beispiele mit anschaulichen Erklärungen.

Im folgenden wird nun Cython durch die `cythonmagic` Extension geladen,
die eingangs vorgestellte Python Funktion nach Cython umgeschrieben,
und in der Zelle mittels des `%%cython` Magic-Commandos ausgeführt.

Von der im Hintergrund ablaufenden Compilierung nach C,
das Linken mit den Python Bibliotheken,
und das anschließende dynamische laden in den aktuellen Python-Kernel bekommt man nichts mit --
es dauert nur ein paar Sekunden, die sich dann später durch die beschleunigte Ausführungszeit wieder leicht einholen lassen ;-)

In [4]:
%load_ext Cython

In [5]:
%%cython
cimport cython
@cython.boundscheck(False)
cpdef f_cy(float x):
    cdef double y = 0.0
    cdef int i
    for 0 <= i <= 100000:
        y += x
    return y

Kontrolle, dass das Ergebnis stimmt und anschließendes Benchmark mit `%timeit`

In [6]:
f_cy(0.01)

1000.0099776480347

In [7]:
%timeit f_cy(0.01)

The slowest run took 4.01 times longer than the fastest. This could mean that an intermediate result is being cached 
10000 loops, best of 3: 95.3 µs per loop


Die `--annotate` Option zeigt in gelb genauer an,
was wo Konvertierungen oder Python-Objekte noch im Spiel sind (die das Programm langam machen):

In [8]:
%%cython --annotate
cimport cython
@cython.boundscheck(False)
cpdef f_cy(float x):
    cdef double y = 0.0
    cdef int i
    for 0 <= i <= 100000:
        y += x
    return y

## Cython & OpenMP

OpenMP ist ein älteres Programmierhilfsmittel, um Schleifen und parallelisierbare Datenstrukturen im Code einfach ausdrücken zu können.
Die Arbeit wird hierfür auf alle CPU-Kerne verteilt.
In dem hier vorgestellten Beispiel ist es die Funktion `prange`, welche diese in Kombination mit der Addition der Ypsilons erledigt.
Implizit wird hier auf die [Nebenläufigkeit](http://de.wikipedia.org/wiki/Nebenl%C3%A4ufigkeit) der Berechnung und dem anschließenden zusammenführen der Ergebnisse aufgepasst!

In [9]:
%%cython --compile-args=-fopenmp --link-args=-fopenmp

cimport cython
from cython.parallel import parallel, prange

@cython.boundscheck(False)
cpdef f_cy_omp(float x):
    cdef double y = 0.0
    cdef Py_ssize_t i
    with nogil:
        for i in prange(100000):
            y += x
    return y

In [10]:
f_cy_omp(0.01)

999.9999776482582

In [11]:
%timeit f_cy_omp(0.01)

The slowest run took 16.92 times longer than the fastest. This could mean that an intermediate result is being cached 
10000 loops, best of 3: 27.1 µs per loop
